In [1]:
%pip install beautifulsoup4
%pip install playwright
# installs playwright for different web browsers using the command line '!'
!playwright install

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

SEASONS = list(range(2016,2023))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [3]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [4]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
for season in SEASONS:
    await scrape_season(season)

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<Connection.run() done, defined at C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
  File "C:\Users\henry\anaconda3\lib\asyncio\subprocess.py", line 236, in create_subprocess_exec
    transport, protocol = await loop.subprocess_exec(
  File "C:\Users\henry\anaconda3\lib\asyncio\base_events.py", line 1676, in subprocess_exec
    transport = await self._make_subprocess_transport(
  File "C:\Users\henry\an

NotImplementedError: 

The following script when given a URL and a selector, will return html components for us to process.

Since playwright opens a browser on the side (which takes time), playwright runs asyncronously. This means that functions are run on another thread and will return when the function is complete.

In [7]:
!python scraper.py

C:\Users\henry\NBA Predictor\scraper.py:58: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 58 of the file C:\Users\henry\NBA Predictor\scraper.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)
Traceback (most recent call last):
  File "C:\Users\henry\NBA Predictor\scraper.py", line 26, in get_html
    await page.goto(url)
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\async_api\_generated.py", line 8992, in goto
    await self._impl_obj.goto(
  File "C:\Users\henry\anaconda3\lib\site-packages\playwright\_impl\_page.py", line 556, in goto
    return await self._main_frame.goto(**locals_to_p

Bulls vs Clippers, November 19, 2016 | Basketball-Reference.com
Hawks vs Knicks, November 20, 2016 | Basketball-Reference.com
Trail Blazers vs Nets, November 20, 2016 | Basketball-Reference.com
Pacers vs Thunder, November 20, 2016 | Basketball-Reference.com
Jazz vs Nuggets, November 20, 2016 | Basketball-Reference.com
Raptors vs Kings, November 20, 2016 | Basketball-Reference.com
Bulls vs Lakers, November 20, 2016 | Basketball-Reference.com
Grizzlies vs Hornets, November 21, 2016 | Basketball-Reference.com
Warriors vs Pacers, November 21, 2016 | Basketball-Reference.com
Heat vs 76ers, November 21, 2016 | Basketball-Reference.com
Suns vs Wizards, November 21, 2016 | Basketball-Reference.com
Rockets vs Pistons, November 21, 2016 | Basketball-Reference.com
Magic vs Bucks, November 21, 2016 | Basketball-Reference.com
Celtics vs Timberwolves, November 21, 2016 | Basketball-Reference.com
Mavericks vs Spurs, November 21, 2016 | Basketball-Reference.com
Raptors vs Clippers, November 21, 2016 |


  File "C:\Users\henry\NBA Predictor\scraper.py", line 29, in get_html
    except PlaywrightTimeout:
NameError: name 'PlaywrightTimeout' is not defined


In [ ]:
#loop through every .html file in the scores directory
#add the file to the relative season folder based on date of the match
#if month lies between 9-12 month n-1 year or 1-7 month n year, then place file in n-1 - n folder